# 50HL30D
![6%](https://progress-bar.dev/6?title=progress&width=100)

#### Description:
- Solving 50 Hard Leetcode problems in 30 days.<br>
- Languange of choice: `Python`<br>

#### Timelines:
- Start Date: 8th Feb 2024
- Challenge End Date: 9th Mar 2024

In [1]:
from typing import List
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    return temp
    
conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)
        return visited

    def __str__(self):
        return str(self.val)

    def from_array(nums: List[int]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)
                
                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

temp = BinaryTreeNode.from_array([1,2,3,4,5,6,None,None,None,7,8])
temp.to_list()

[1, 2, 3, 4, 5, 6, None, None, None, 7, 8, None, None, None, None, None, None]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr
            
        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

# 1. 
Date: 08th Feb 2024<br>
Hard Leetcode: reducing-dishes<br>
Solution: https://leetcode.com/problems/reducing-dishes/submissions/1169321043/

In [4]:
class Solution:
    def maxSatisfaction(self, satisfaction: List[int]) -> int:
        # Compute once
        N = len(satisfaction)

        # Sort to find the best possible time for each (order wise)
        satisfaction.sort()

        @functools.cache
        def backtrack(currIdx: int = 0, time: int = 1) -> int:
            if currIdx == N:
                return 0
            else:
                return max(
                    (time * satisfaction[currIdx]) + backtrack(currIdx + 1, time + 1),
                    backtrack(currIdx + 1, time)
                )

        return backtrack()
    
# Testing the solution
assert Solution().maxSatisfaction([-1, -8, 0, 5, -9]) == 14
assert Solution().maxSatisfaction([4, 3, 2]) == 20
assert Solution().maxSatisfaction([-1, -2, -3]) == 0

# 2. 
Date: 09th Feb 2024<br>
Hard Leetcode: minimum-one-bit-operations-to-make-integers-zero<br>
Solution: https://leetcode.com/problems/minimum-one-bit-operations-to-make-integers-zero/submissions/1170668823/

In [5]:
class Solution:
    def minimumOneBitOperations__(self, n: int) -> int:
        def changeBit(b: str, idx: int):
            bits_ = list(b)
            curr = bits_[idx]
            curr = str(int(not bool(int(curr))))
            bits_[idx] = curr
            return ''.join(bits_)
        
        def op1(b: str, N: int):
            'Last bit is changed'
            return changeBit(b, N - 1)

        def op2(b: str, N: int):
            """
            Swap ith bit given i-1st bit is 1 and all bits to the right are 0
            """
            i = b.rindex('1')
            if i >= 1:
                return changeBit(b, i - 1)
            else:
                return b

        def backtrack(b: str, processed: set[str] = set()):
            if not int(b):
                return 0
            else:
                op1_result = op1(b, N)
                op2_result = op2(b, N)
                op1_count = op2_count = math.inf
                if op1_result not in processed: 
                    op1_count = backtrack(op1_result, {*processed, b})
                if op2_result not in processed: 
                    op2_count = backtrack(op2_result, {*processed, b})

                result = 1 + min(op1_count, op2_count)
                return result

        bits = bin(n)[2:]
        N = len(bits)
        return backtrack(bits)

    def minimumOneBitOperations_(self, n: int) -> int:
        if n == 0:
            return 0

        bits = bin(n)[2:]
        bit_length = len(bits)
        maxN = (2 ** bit_length) - 1
        minN = 2 ** (bit_length - 1)
        dp = [_ for _ in range(minN, maxN + 1)]
        
        def splitAndMerge(arr: list[int], rev=False):
            N = len(arr)
            if N == 1:
                return arr
            else:
                mid = N // 2
                left = splitAndMerge(arr[:mid], rev=False)
                right = splitAndMerge(arr[mid:], rev=True)
                return right + left if rev else left + right

        dp = splitAndMerge(dp, rev=True)
        return dp[n - minN]

    def minimumOneBitOperations(self, n: int) -> int:
        # https://www.youtube.com/watch?v=yRI18_MaG7k
        if n == 0:
            return 0
        else:
            k = 0
            while 2 ** k <= n:
                k += 1 
            k -= 1
            return 2 ** (k + 1) - 1 - self.minimumOneBitOperations(2 ** k ^ n)

# Testing the solution
for _ in range(1, 300):
    assert Solution().minimumOneBitOperations_(_) == Solution().minimumOneBitOperations(_), f"failed for {_}"

# 3. 
Date: 10th Feb 2024<br>
Hard Leetcode: human-traffic-of-stadium<br>
Solution: https://leetcode.com/problems/human-traffic-of-stadium/submissions/1171130705

In [6]:
temp = read_lc_df("""
+------+------------+-----------+
| id   | visit_date | people    |
+------+------------+-----------+
| 1    | 2017-01-01 | 10        |
| 2    | 2017-01-02 | 109       |
| 3    | 2017-01-03 | 150       |
| 4    | 2017-01-04 | 99        |
| 5    | 2017-01-05 | 145       |
| 6    | 2017-01-06 | 1455      |
| 7    | 2017-01-07 | 199       |
| 8    | 2017-01-09 | 188       |
+------+------------+-----------+
""")

temp['visit_date'] = pd.to_datetime(temp['visit_date'])

def human_traffic_df(stadium: pd.DataFrame) -> pd.DataFrame:
    df = stadium[stadium['people'] >= 100]
    ids = set(df['id'])
    result = []
    for row in df.itertuples():
        row = row[1:]
        curr_id = row[0]
        if curr_id - 1 in ids and curr_id - 2 in ids:
            result.append(row)
        elif curr_id - 1 in ids and curr_id + 1 in ids:
            result.append(row)
        elif curr_id + 1 in ids and curr_id + 2 in ids:
            result.append(row)

    df = pd.DataFrame.from_records(result, columns=stadium.columns)
    df.columns = stadium.columns
    return df

# Pandas / python solution
display(human_traffic_df(temp))

# Write to SQL DB
temp.to_sql("temp", if_exists='replace', con=conn, index=False)

display(pd.read_sql("""
    with cte1 as (
        select *,  id - row_number() over(order by id asc) as rnk
        from temp
        where people >= 100
    ),

    cte2 as (
        select c.*, count(*) over (partition by rnk) rnk_count from cte1 c order by c.visit_date
    )

    select c.id, c.visit_date, c.people from cte2 c where c.rnk_count > 2
""", con=conn))

display(pd.read_sql("""
    with cte1 as (
        select * from temp where people >= 100
    ),

    cte2 as (
        select 
                c.*, 
                ifnull(-c.id - 1 + lead(id, 1) over (), 1) = 0 as lead1,
                ifnull(-c.id - 2 + lead(id, 2) over (), 1) = 0 as lead2,
                ifnull(c.id - 1 - lag(id, 1) over (), 1) = 0 as lag1,
                ifnull(c.id - 2 - lag(id, 2) over (), 1) = 0 as lag2
        from cte1 c
    ),

    cte3 as (
        select 
            c.*,  
            (lead1 + lead2 + lag1 + lag2) as filter
        from cte2 c
    )

    select c.id, c.visit_date, c.people from cte3 c where c.filter >= 2
""", con=conn))

,id,visit_date,people
0,5.0,2017-01-05,145.0
1,6.0,2017-01-06,1455.0
2,7.0,2017-01-07,199.0
3,8.0,2017-01-09,188.0


,id,visit_date,people
0,5.0,2017-01-05T00:00:00,145.0
1,6.0,2017-01-06T00:00:00,1455.0
2,7.0,2017-01-07T00:00:00,199.0
3,8.0,2017-01-09T00:00:00,188.0


,id,visit_date,people
0,5.0,2017-01-05T00:00:00,145.0
1,6.0,2017-01-06T00:00:00,1455.0
2,7.0,2017-01-07T00:00:00,199.0
3,8.0,2017-01-09T00:00:00,188.0


# 4. 
Date: 10th Feb 2024<br>
Hard Leetcode: largest-rectangle-in-histogram<br>
Solution: https://leetcode.com/problems/largest-rectangle-in-histogram/submissions/1171340811

In [28]:
class Solution:
    def largestRectangleArea(self, heights: List[int]) -> int:
        """
        For each hist, count the number of adjacent hist less than or equal to itself
        Travel left to right, find the next smallest number, repeat by travelling from left to right
        """ 
        N = len(heights)
        nge_rtl, nge_ltr = [N - 1 for _ in range(N)], [0 for _ in range(N)]
        stack = []
        for i in range(N):
            while stack and heights[stack[-1]] > heights[i]:
                nge_rtl[stack.pop()] = i - 1
            stack.append(i)
        
        stack = []
        for i in range(N - 1, -1, -1):
            while stack and heights[stack[-1]] > heights[i]:
                nge_ltr[stack.pop()] = i + 1
            stack.append(i)

        maxArea = -math.inf
        for i in range(N):
            curr = heights[i]
            curr = (nge_rtl[i] - nge_ltr[i] + 1) * curr
            maxArea = max(maxArea, curr)

        return maxArea

# Testingt the soltuion
assert Solution().largestRectangleArea([2, 1, 5, 6, 2, 3]) == 10
assert Solution().largestRectangleArea([2,4]) == 4